In [ ]:
import sys
import os
import lmdb  # install lmdb by "pip install lmdb"
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import json
from skimage import io, data, measure, color
from torchvision import transforms
import PIL
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
import random
import torch
import lmdb
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import itertools
from data_loaders import *
import pickle
import scipy.misc
import pandas as pd
import utils
import imgaug as ia
from imgaug import augmenters as iaa
from dataset_utils import *
% matplotlib inline




ModuleNotFoundError: No module named 'data_loaders'

In [8]:
datafolder = '/home/filippos/data/MSR-Demosaicing/Dataset_LINEAR_without_noise/'
demosaic_dataset = MSRDemosaicDataset(root_dir=datafolder+'bayer_panasonic/',
                                      selection_file=datafolder+'bayer_panasonic/train.txt',
                                      )
    
tmp_save = '/home/filippos/data/tmp/'
files = glob.glob(tmp_save+'*.pkl')
for f in files:
    os.remove(f)
datafolder = [datafolder+f for f in os.listdir(datafolder)]

In [10]:
max_translation = 10 # pixels
max_angle = 2 # degrees
burst_size = 8 # number of bursts

for data_file in demosaic_dataset:
    img_gt = data_file['image_gt']
    img_input = data_file['image_input']
    img_input_compress = np.zeros((int(img_input.shape[0]/2),int(img_input.shape[1]/2),4))
    img_input_compress[...,0] = img_input[::2,::2,0]
    img_input_compress[...,1] = img_input[::2,1::2,1]
    img_input_compress[...,2] = img_input[1::2,::2,1]
    img_input_compress[...,3] = img_input[1::2,1::2,2]

    tr_x, tr_y = max_translation/img.shape[0], max_translation/img.shape[1]
    seq = iaa.Sequential(
        [iaa.Affine(
                translate_percent={"x": (-tr_x, tr_x), "y": (-tr_y, tr_y)}, # translate by -max_translation to +max_translation percent (per axis)
                rotate=(-max_angle, max_angle), # rotate by -max_angle to +max_angle degrees
                order=3, # use bicubic
                mode='symmetric' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )]
    )
    images_aug = seq.augment_images([img_gt]*(burst_size-1))
    images_aug.append(img_gt)
    images_aug = np.array(images_aug)


    images_aug = images_aug[:,2:-2,46:-46,:]
    img_gt = img_gt[2:-2,46:-46,:]
    mask = np.zeros_like(images_aug)
    mask[:,::2,::2,0] = 1
    mask[:,::2,1::2,1] = 1
    mask[:,1::2,::2,1] = 1
    mask[:,1::2,1::2,2] = 1
    images_aug *= mask
    json_burst = {}
    json_burst['image_gt'] = img_gt
    json_burst['image_input'] = images_aug
    json_burst['filename'] = data_file['filename']
    pickle.dump(json_burst,open(tmp_save+'/'+json_burst['filename']+'.pkl','wb'))

In [12]:
file_list = glob.glob(tmp_save+'*.pkl')

class JSONDataset(Dataset):
    """JSON dataset."""

    def __init__(self, path):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file_list = glob.glob(path+'*.pkl')

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        sample = pickle.load(open(self.file_list[idx],'rb'))
        try:
            warp_matrix = calculate_affine_matrices(sample['image_input'])
            if warp_matrix is None: return None
            sample['warp_matrix'] = warp_matrix
            return sample
        except Exception as e:
            print(e)
            return None

In [13]:
ds = JSONDataset(tmp_save)
len(ds)

600

In [15]:
def store_files(lmdb_dataset, dataset):
    cnt = 0
    cache = {}
    nSamples = len(dataset)
    for i, sample in enumerate(dataset):
        if sample is None:
            print('Lost One')
            continue
        if not np.array_equal(sample['warp_matrix'][-1],np.array([[1,0,0],[0,1,0]])):
            print('error')
        sample_json = json.dumps(sample, cls=NumpyEncoder)
        jsonKey = 'json-%09d' % cnt
        cache[jsonKey.encode()] = sample_json.encode()
        if cnt % 30 == 0:
            writeCache(lmdb_dataset, cache)
            cache = {}
            print('Written %d / %d' % (cnt, nSamples))
        cnt += 1
    nSamples = cnt
    cache['num-samples'.encode()] = str(nSamples).encode()
    writeCache(lmdb_dataset, cache)
    print('Done!')
    

class lmdbDataset(Dataset):

    def __init__(self, lmdb_path, transform=None):
        self.env = lmdb.open(lmdb_path,
                 max_readers=100,
                 readonly=True,
                 lock=False,
                 readahead=False,
                 meminit=False)
        self.txn = self.env.begin(write=False) 
        self.nSamples = int(self.txn.get('num-samples'.encode()))
        self.indices = range(self.nSamples) 
        self.transform = transform

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        jsonKey = 'json-%09d' % (index)
        sample = self.txn.get(jsonKey.encode())
        sample = json.loads(sample)
        for k in sample.keys():
            if type(sample[k]) is list:
                sample[k] = np.array(sample[k])
        if self.transform:
            sample = self.transform(sample)

        return sample

def lmdb_nsamples(db):
    env = lmdb.open(db,
                max_readers=1,
                readonly=True,
                lock=False,
                readahead=False,
                meminit=False)

    with env.begin(write=False) as txn:
        nSamples = int(txn.get('num-samples'.encode()))
    return nSamples

In [17]:
outputPath = '/home/filippos/data/MSR_burst_train_v3_augmented.lmdb'
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
env = lmdb.open(outputPath, map_size=1048576000*10)
store_files(env, ds)
env.close()
print(lmdb_nsamples(outputPath))

Written 0 / 600
Written 30 / 600
Written 60 / 600
Written 90 / 600
Written 120 / 600
Written 150 / 600
Written 180 / 600
Written 210 / 600
Written 240 / 600
Written 270 / 600
0 /io/opencv/modules/video/src/ecc.cpp:540: error: (-7) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function findTransformECC

1 /io/opencv/modules/video/src/ecc.cpp:525: error: (-7) NaN encountered. in function findTransformECC

Lost One
Written 300 / 600
Written 330 / 600
Written 360 / 600
0 /io/opencv/modules/video/src/ecc.cpp:540: error: (-7) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function findTransformECC

1 /io/opencv/modules/video/src/ecc.cpp:540: error: (-7) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function findTransformECC

Lo